In [ ]:
#  |0> -- Z --> |0>
#  |0> -- X --> |+> or |->
#
#  |-> -- Z --> |0> or |1>
#  |-> -- X --> |->


In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
import numpy as np
import qiskit
%matplotlib inline

In [2]:
# generate one qubit

q = QuantumRegister(1,'q')
c = ClassicalRegister(2,'c')

# Alice
Alice_circ0 = QuantumCircuit(q,c)  # |0>
Alice_circ0.barrier(q)


Alice_circ1 = QuantumCircuit(q,c)  # |->
Alice_circ1.x(q[0])
Alice_circ1.h(q[0])
Alice_circ1.barrier(q)

In [3]:
# Bob measurement
B1 = QuantumCircuit(q,c) # Z basis
B1.measure(q[0],c[0])

B2 = QuantumCircuit(q,c) # X basis
B2.h(q[0])
B2.measure(q[0],c[0])

In [4]:
n=32   # n qubits

import random
def random_01str(list1,num):
    for i in range(num):
        list1.append(random.randint(0,1))

        
Alice_01=[]
Bob_basis=[]

Alice_01.clear()
Bob_basis.clear()

random_01str(Alice_01,n)
random_01str(Bob_basis,n)

print("Alice_01 0(0)(|0>) or 1(1)(|->) : {}".format(Alice_01))
print("Bob_basis B1(0)(Z) or B2(1)(X) : {}".format(Bob_basis))

Alice_01 0(0)(|0>) or 1(1)(|->) : [0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0]
Bob_basis B1(0)(Z) or B2(1)(X) : [1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]


In [5]:
# generate circuits

Alice_circ = [Alice_circ0,Alice_circ1]
Bob_circ = [B1,B2]

circuits=[]
circuits.clear()

for i in range(n):
    if not Alice_01[i]:
        OneCircuit = Alice_circ[0] + Bob_circ[Bob_basis[i]]
    elif Alice_01[i]:
        OneCircuit = Alice_circ[1] + Bob_circ[Bob_basis[i]]

    circuits.append(OneCircuit)
    
    
circuits[0].draw()

In [6]:
from qiskit import execute, BasicAer
from qiskit.tools.visualization import plot_histogram
backend_sim = BasicAer.get_backend('qasm_simulator')

job_sim = qiskit.execute(circuits, backend_sim, shots=1)
result_sim = job_sim.result()
counts = result_sim.get_counts(circuits[0])
print(counts)

{'00': 1}


In [7]:
# get Bob's result

Bob_result=[]
Bob_result.clear()

for i in range(n):
    Bob_result.append(int((list(result_sim.get_counts(circuits[i]))[0])[1]))
    
print("Bob_result   = {}".format(Bob_result))

Bob_result   = [0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0]


In [8]:
#Check the state that can be ensured

ensured_pos=[]
ensured_state=[]
Bob_key=[]

ensured_pos.clear()
ensured_state.clear()
Bob_key.clear()

for i in range(n):
    if Bob_basis[i]==0 and Bob_result[i]==1: # |1>
        ensured_pos.append(i)
        ensured_state.append(1)
        Bob_key.append(Bob_basis[i])
    elif Bob_basis[i]==1 and Bob_result[i]==0: # |+>
        ensured_pos.append(i)
        ensured_state.append(1)
        Bob_key.append(Bob_basis[i])
    else:
        ensured_state.append(0)

print("ensured_pos = {}".format(ensured_pos))
print("ensured_state = {}".format(ensured_state))
print("Bob_key = {}".format(Bob_key))

ensured_pos = [0, 4, 5, 15, 19, 25, 31]
ensured_state = [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
Bob_key = [1, 0, 1, 0, 1, 0, 1]


In [9]:
#Bob informs the position of states that can be ensured to Alice,
#and Alice gets key

Alice_key=[]

Alice_key.clear()

for i in range(n):
    if ensured_state[i]==1:
        Alice_key.append(int(not Alice_01[i]))


print("Alice_key = {}".format(Alice_key))

Alice_key = [1, 0, 1, 0, 1, 0, 1]
